In [34]:
import pandas as pd

ratings = pd.read_csv('data/ml-32m/ratings.csv', dtype={'userId': 'int64', 'movieId': 'int64', 'rating': 'float64'})
movies = pd.read_csv('data/ml-32m/movies.csv', dtype={'movieId': 'int64', 'title': 'str', 'genres': 'str'})

In [18]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [19]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
print(ratings.isnull().sum())
print()
print(movies.isnull().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

movieId    0
title      0
genres     0
dtype: int64


In [49]:
data = pd.merge(ratings, movies, on='movieId')
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,17,4.0,944249077,Sense and Sensibility (1995),Drama|Romance
1,1,25,1.0,944250228,Leaving Las Vegas (1995),Drama|Romance
2,1,29,2.0,943230976,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
3,1,30,5.0,944249077,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama
4,1,32,5.0,943228858,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [28]:
print(ratings['userId'].nunique(), "users")
print(ratings['movieId'].nunique(), "movies")

200948 users
84432 movies


In [50]:
from scipy.sparse import csr_matrix

user_movie_matrix_sparse = csr_matrix((ratings['rating'], 
                                        (ratings['userId'], 
                                         ratings['movieId'])))
user_movie_matrix_sparse

<200949x292758 sparse matrix of type '<class 'numpy.float64'>'
	with 32000204 stored elements in Compressed Sparse Row format>

In [64]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

trainSet, testSet = train_test_split(data, test_size=0.2, random_state=27)

model = SVD()
model.fit(trainSet)

predictions = model.test(testSet)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 0.7715
RMSE: 0.7715408130210513


In [75]:
custom_user_genres = ['Action']
liked_movies = [114662]

filtered_movies = movies[movies['genres'].str.contains('|'.join(custom_user_genres))]

recommended_movies = []
for movie_id in filtered_movies['movieId'].values:
    predicted_rating = model.predict(uid=0, iid=movie_id)
    recommended_movies.append((movie_id, predicted_rating.est))

recommended_movies.sort(key=lambda x: x[1], reverse=True)

top_recommendations = filtered_movies[filtered_movies['movieId'].isin([x[0] for x in recommended_movies[:10]])]
print(top_recommendations[['title','genres']])

       movieId                   title
22394   114662  American Sniper (2014)
                                                   title  \
1930         Seven Samurai (Shichinin no samurai) (1954)   
2867                                   Fight Club (1999)   
8585   Crooks in Clover (a.k.a. Monsieur Gangster) (L...   
9179   Revolutionary Girl Utena: Adolescence of Utena...   
35517                 Hornblower: The Even Chance (1998)   
46103                            Band of Brothers (2001)   
48042                        I Am So Proud of You (2008)   
70818                                     Firefly (2002)   
76413                             Attack On Titan (2013)   
85883         Spider-Man: Across the Spider-Verse (2023)   

                                                  genres  
1930                              Action|Adventure|Drama  
2867                         Action|Crime|Drama|Thriller  
8585                                 Action|Comedy|Crime  
9179   Action|Adventure|A

In [76]:
import pickle

with open('movie_recommendations.pkl', 'wb') as file:
    pickle.dump(recommended_movies, file)
    
print(f"Model saved!")

Model saved!
